<a href="https://colab.research.google.com/github/ImdadKhan0266/CSML-1010/blob/main/Project_Saira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Fetching

In [1]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
import os
import polars as pl
import plotly.express as px
pl.Config.set_fmt_float("full") # to get rid of scientific notation while printing stuff
pl.Config.set_tbl_width_chars(1000)


polars.config.Config

In [2]:
# make sure you are storing your tmdb data in this directory
os.makedirs('/content/drive/MyDrive/project_dva', exist_ok=True)
# Change directory
os.chdir('/content/drive/MyDrive/project_dva')
# Fetching our tmdb data
df = pl.read_parquet('tmdb_clean_data.parquet')

In [3]:
df_orig = df.clone()

In [4]:
# with pl.Config(tbl_cols=-1):
#   print(df.head(5))
df.head()

id,title,original_title,release_date,popularity,vote_average,vote_count,budget,revenue,runtime,status,tagline,homepage,imdb_id,overview,adult,video,original_language,poster_path,backdrop_path,genres,production_companies,production_countries,spoken_languages,genre_Drama,genre_Documentary,genre_Comedy,genre_Horror,genre_Thriller,genre_Romance,genre_Action,genre_Music,genre_TV Movie,genre_Animation,genre_Science Fiction,genre_Crime,genre_Family,…,Writer_name_3,Writer_name_3_popularity,Writer_name_4,Writer_name_4_popularity,Writer_name_5,Writer_name_5_popularity,Musician_name_1,Musician_name_1_popularity,Musician_name_2,Musician_name_2_popularity,Musician_name_3,Musician_name_3_popularity,Musician_name_4,Musician_name_4_popularity,Musician_name_5,Musician_name_5_popularity,movie_idcredits_df,Actor_1_name,Actor_1_popularity,Actor_2_name,Actor_2_popularity,Actor_3_name,Actor_3_popularity,Actor_4_name,Actor_4_popularity,Actor_5_name,Actor_5_popularity,Actor_6_name,Actor_6_popularity,Actor_7_name,Actor_7_popularity,Actor_8_name,Actor_8_popularity,Actor_9_name,Actor_9_popularity,Actor_10_name,Actor_10_popularity
i64,str,str,str,f64,f64,i64,i64,i64,i64,str,str,str,str,str,bool,bool,str,str,str,list[str],list[str],str,list[str],i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64
98,"""Gladiator""","""Gladiator""","""2000-05-04""",5.174,8.218,19470,103000000,465361176,155,"""Released""","""What we do in life echoes in e…","""""","""tt0172495""","""After the death of Emperor Mar…",false,false,"""en""","""/ty8TGRuvJLPUmAR1H1nRIsgwvim.j…","""/Ar7QuJ7sJEiC0oP3I8fKBKIQD9u.j…","[""Action"", ""Drama"", ""Adventure""]","[""Universal Pictures"", ""Scott Free Productions"", … ""DreamWorks Pictures""]","""[""United Kingdom"", ""United Sta…","[""English""]",1,0,0,0,0,0,1,0,0,0,0,0,0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,98,"""Russell Crowe""",0.088,"""Joaquin Phoenix""",0.086,"""Connie Nielsen""",0.082,"""Tommy Flanagan""",0.052,"""Derek Jacobi""",0.043,"""Djimon Hounsou""",0.04,"""Spencer Treat Clark""",0.037,"""Ralf Moeller""",0.034,"""Omid Djalili""",0.031,"""Sven-Ole Thorsen""",0.029
77,"""Memento""","""Memento""","""2000-10-11""",4.971,8.176,15144,9000000,40047236,113,"""Released""","""Some memories are best forgott…","""""","""tt0209144""","""Leonard Shelby is tracking dow…",false,false,"""en""","""/fKTPH2WvH8nHTXeBYBVhawtRqtR.j…","""/7Wev9JMo6R5XAfz2KDvXb7oPMmy.j…","[""Mystery"", ""Thriller""]","[""Newmarket Films"", ""Summit Entertainment"", … ""I Remember Productions""]","""[""United States of America""]""","[""English""]",0,0,0,0,1,0,0,0,0,0,0,0,0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,77,"""Guy Pearce""",0.105,"""Carrie-Anne Moss""",0.05,"""Thomas Lennon""",0.05,"""Joe Pantoliano""",0.046,"""Stephen Tobolowsky""",0.043,"""Mark Boone Junior""",0.037,"""Callum Keith Rennie""",0.035,"""Jorja Fox""",0.033,"""Marianne Muellerleile""",0.029,"""Harriet Sansom Harris""",0.025
9741,"""Unbreakable""","""Unbreakable""","""2000-11-22""",4.907,7.1,9389,75000000,248118121,106,"""Released""","""Are you ready for the truth?""","""""","""tt0217869""","""An ordinary man makes an extra…",false,false,"""en""","""/mLuehrGLiK5zFCyRmDDOH6gbfPf.j…","""/A7CYNTa3fWyU0t207XiecgriQv5.j…","[""Thriller"", ""Drama"", ""Mystery""]","[""Barry Mendel Productions"", ""Blinding Edge Pictures"", ""Touchstone Pictures""]","""[""United States of America""]""","[""English""]",1,0,0,0,1,0,0,0,0,0,0,0,0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,9741,"""Samuel L. Jackson""",0.158,"""Bruce Willis""",0.121,"""Robin Wright""",0.067,"""Spencer Treat Clark""",0.037,"""Michael Kelly""",0.03,"""Eamonn Walker""",0.029,"""Damian Young""",0.026,"""Chance Kelly""",0.025,"""James Handy""",0.025,"""Leslie Stefanson""",0.02

# Data Sanity and Cleaning

In [5]:
# checking for nulls
null_counts = df.select([
    (pl.col(col).is_null().sum()/ pl.len() * 100).round(2).alias(f"{col}_null_count") for col in df.columns
])
with pl.Config(tbl_cols=-1):
  print(null_counts)

shape: (1, 121)
┌────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬

17% nulls in lead actor. seems strange

In [6]:
# Getting numeric columns for some EDA
numeric_columns = [col for col in df.columns if df[col].dtype not in (pl.Utf8, pl.Categorical, pl.List)]
numeric_columns.remove('id') # id not needed
numeric_columns

['popularity',
 'vote_average',
 'vote_count',
 'budget',
 'revenue',
 'runtime',
 'adult',
 'video',
 'genre_Drama',
 'genre_Documentary',
 'genre_Comedy',
 'genre_Horror',
 'genre_Thriller',
 'genre_Romance',
 'genre_Action',
 'genre_Music',
 'genre_TV Movie',
 'genre_Animation',
 'genre_Science Fiction',
 'genre_Crime',
 'genre_Family',
 'genre_Mystery',
 'genre_Adventure',
 'company_BBC',
 'company_Lifetime',
 'company_Universal Pictures',
 'company_Reel One Entertainment',
 'company_Warner Bros. Pictures',
 'company_HBO Documentary Films',
 'company_Lionsgate',
 'company_MarVista Entertainment',
 'company_The Asylum',
 'company_Paramount Pictures',
 'language_English',
 'language_Español',
 'language_Français',
 'language_Deutsch',
 'language_No Language',
 'language_Italiano',
 'language_',
 'language_Português',
 'language_Pусский',
 'language_العربية',
 'movie_id',
 'Producer_name_1_popularity',
 'Producer_name_2_popularity',
 'Producer_name_3_popularity',
 'Producer_name_4_pop

In [7]:

# Checking 0 count here
zero_counts = df.select(numeric_columns).select([
    (pl.col(col).eq(0).sum() / pl.len() * 100).round(2).alias(f"{col}")  for col in numeric_columns
])

zero_counts



popularity,vote_average,vote_count,budget,revenue,runtime,adult,video,genre_Drama,genre_Documentary,genre_Comedy,genre_Horror,genre_Thriller,genre_Romance,genre_Action,genre_Music,genre_TV Movie,genre_Animation,genre_Science Fiction,genre_Crime,genre_Family,genre_Mystery,genre_Adventure,company_BBC,company_Lifetime,company_Universal Pictures,company_Reel One Entertainment,company_Warner Bros. Pictures,company_HBO Documentary Films,company_Lionsgate,company_MarVista Entertainment,company_The Asylum,company_Paramount Pictures,language_English,language_Español,language_Français,language_Deutsch,language_No Language,language_Italiano,language_,language_Português,language_Pусский,language_العربية,movie_id,Producer_name_1_popularity,Producer_name_2_popularity,Producer_name_3_popularity,Producer_name_4_popularity,Producer_name_5_popularity,Director_name_1_popularity,Director_name_2_popularity,Director_name_3_popularity,Director_name_4_popularity,Director_name_5_popularity,Writer_name_1_popularity,Writer_name_2_popularity,Writer_name_3_popularity,Writer_name_4_popularity,Writer_name_5_popularity,Musician_name_1_popularity,Musician_name_2_popularity,Musician_name_3_popularity,Musician_name_4_popularity,Musician_name_5_popularity,movie_idcredits_df,Actor_1_popularity,Actor_2_popularity,Actor_3_popularity,Actor_4_popularity,Actor_5_popularity,Actor_6_popularity,Actor_7_popularity,Actor_8_popularity,Actor_9_popularity,Actor_10_popularity
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,0.37,0,85.78,92.88,4.73,100,100,73.88,76.76,80.11,85.78,87.6,92.26,92.96,93.55,93.73,94.47,94.52,94.99,95.48,95.87,96.25,99.11,99.53,99.57,99.6,99.65,99.66,99.67,99.68,99.71,99.72,29.97,97.28,97.51,98.32,98.92,99.01,99.15,99.3,99.31,99.43,0,0,0,0,0,0,0.01,0,0,0,0,0.01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.01,0,0,0,0.01


**Too many 0s. removing data that has 0 vote_average**

In [14]:
df = df.filter(pl.col("vote_average")>0)
df.shape

(107978, 121)

In [17]:
# Checking summary of numeric columns

desc = df.select(numeric_columns).describe()
desc = pl.DataFrame(desc)
summary_df_rounded = desc.with_columns([
    pl.col(pl.Float64).round(2)  # Round all float64 columns to 2 decimal places
])


(summary_df_rounded)



statistic,popularity,vote_average,vote_count,budget,revenue,runtime,adult,video,genre_Drama,genre_Documentary,genre_Comedy,genre_Horror,genre_Thriller,genre_Romance,genre_Action,genre_Music,genre_TV Movie,genre_Animation,genre_Science Fiction,genre_Crime,genre_Family,genre_Mystery,genre_Adventure,company_BBC,company_Lifetime,company_Universal Pictures,company_Reel One Entertainment,company_Warner Bros. Pictures,company_HBO Documentary Films,company_Lionsgate,company_MarVista Entertainment,company_The Asylum,company_Paramount Pictures,language_English,language_Español,language_Français,…,language_Italiano,language_,language_Português,language_Pусский,language_العربية,movie_id,Producer_name_1_popularity,Producer_name_2_popularity,Producer_name_3_popularity,Producer_name_4_popularity,Producer_name_5_popularity,Director_name_1_popularity,Director_name_2_popularity,Director_name_3_popularity,Director_name_4_popularity,Director_name_5_popularity,Writer_name_1_popularity,Writer_name_2_popularity,Writer_name_3_popularity,Writer_name_4_popularity,Writer_name_5_popularity,Musician_name_1_popularity,Musician_name_2_popularity,Musician_name_3_popularity,Musician_name_4_popularity,Musician_name_5_popularity,movie_idcredits_df,Actor_1_popularity,Actor_2_popularity,Actor_3_popularity,Actor_4_popularity,Actor_5_popularity,Actor_6_popularity,Actor_7_popularity,Actor_8_popularity,Actor_9_popularity,Actor_10_popularity
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,107978,…,107978,107978,107978,107978,107978,100345,51803,28927,14814,7273,3407,98799,9999,1394,638,449,57025,15596,3579,1033,446,1125,504,355,266,198,90902,89081,79123,72310,65733,58950,52781,47421,42983,38798,35123
"""null_count""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,7633,56175,79051,93164,100705,104571,9179,97979,106584,107340,107529,50953,92382,104399,106945,107532,106853,107474,107623,107712,107780,17076,18897,28855,35668,42245,49028,55197,60557,64995,69180,72855
"""mean""",0.61,5.96,145.66,1896985.81,4816740.28,65.32,0,0,0.26,0.23,0.2,0.14,0.12,0.08,0.07,0.06,0.06,0.06,0.05,0.05,0.05,0.04,0.04,0.01,0,0,0,0,0,0,0,0,0,0.7,0.03,0.02,…,0.01,0.01,0.01,0.01,0.01,524026.6,0.01,0,0,0,0,0.01,0,0,0,0,0.01,0,0,0,0,0,0,0,0,0,516503.88,0.31,0.09,0.05,0.04,0.03,0.02,0.02,0.01,0.01,0.01
"""std""",1.09,2.14,1001.88,13252227.13,48819186.93,68.66,null,null,0.44,0.42,0.4,0.35,0.33,0.27,0.26,0.25,0.24,0.23,0.23,0.22,0.21,0.2,0.19,0.09,0.07,0.07,0.06,0.06,0.06,0.06,0.06,0.05,0.05,0.46,0.16,0.16,…,0.1,0.09,0.08,0.08,0.08,372638.09,0.25,0.02,0.01,0,0,0.21,0.03,0.05,0,0,0.17,0.07,0.01,0,0,0,0,0,0,0,379276.92,4.03,1.44,0.85,0.59,0.48,0.39,0.3,0.28,0.23,0.19
"""min""",0,0.5,1,0,-12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0,0,0
"""25%""",0.05,4.9,1,0,0,19,null,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,222159,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,199966,0,0,0,0,0,0,0,0,0,0
"""50%""",0.11,6,3,0,0,80,null,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,…,0,0,0,0,0,466306,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,456659,0.01,0.01,0,0,0,0,0,0,0,0
"""75%""",0.43,7.1,12,0,0,93,null,null,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,…,0,0,0,0,0,779796,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,781188,0.03,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01
"""max""",7.68,10,37197,460000000,2923706026,14400,0,0,1,1,1,1,1,1,1,1,1,1,1,1,

In [18]:
# Checking empty values in character columns
char_columns = [col for col in df.columns if df[col].dtype in (pl.Utf8, pl.Categorical)]


empty_counts = df.select(char_columns).select([
    (pl.col(col).eq("").sum() / pl.len() * 100).round(2).alias(f"{col}")  for col in char_columns
])

empty_counts

title,original_title,release_date,status,tagline,homepage,imdb_id,overview,original_language,poster_path,backdrop_path,production_countries,Producer_name_1,Producer_name_2,Producer_name_3,Producer_name_4,Producer_name_5,Director_name_1,Director_name_2,Director_name_3,Director_name_4,Director_name_5,Writer_name_1,Writer_name_2,Writer_name_3,Writer_name_4,Writer_name_5,Musician_name_1,Musician_name_2,Musician_name_3,Musician_name_4,Musician_name_5,Actor_1_name,Actor_2_name,Actor_3_name,Actor_4_name,Actor_5_name,Actor_6_name,Actor_7_name,Actor_8_name,Actor_9_name,Actor_10_name
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,0,0,0,63.61,68.56,2.92,0.97,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#EDA

In [19]:
# EDA Plots
agg_df = df.group_by("production_countries").agg([
    pl.col("id").count().alias("count"),
    (pl.col("vote_average").eq(0).sum() / pl.count() * 100).alias("vote_average_zeros_percentage")
])
agg_df=agg_df.sort("count", descending=True)
#agg_df = agg_df.filter(pl.col("count")>10000) #pick only rows with count over 1k
agg_df = agg_df.head(10)
pandas_df = agg_df.to_pandas()

# Create a bar plot using Plotly
fig = px.bar(pandas_df, x="production_countries", y="count", title="Count by production_countries")
fig.show()

#fig2 = px.bar(pandas_df, x="production_countries", y="vote_average_zeros_percentage", title="vote_average zeros percentage by production_countries")
#fig2.show()



<ipython-input-19-df133bae3750>:4: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  (pl.col("vote_average").eq(0).sum() / pl.count() * 100).alias("vote_average_zeros_percentage")


**Why so many empty values in production_countries**

In [ ]:
# Check correlation of vote average with revenue
df_filtered = df.filter((pl.col("vote_average") > 0) & (pl.col("revenue") > 0) & (pl.col("budget") > 0) & (pl.col("vote_count") > 100))
correlation = df_filtered.select([
    pl.col("vote_average"),
    pl.col("revenue")/pl.col("budget")
]).corr()
correlation


vote_average,revenue
f64,f64
1,0.02189155528518268
0.02189155528518268,1


Run a baseline model on the data.

# Baseline Model

In [20]:
# Step 0: Import libraries
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import ast


pd.set_option('display.max_rows', 150)

In [10]:

# Step 1: Convert Polars DataFrame to pandas DataFrame
df_pd = df.to_pandas()
# df_pd.dtypes

In [22]:
def parse_as_list(x):
    if isinstance(x, str):
        try:
            # Attempt to parse the string into a Python object
            parsed = ast.literal_eval(x)
            # Only return it if it’s truly a list
            if isinstance(parsed, list):
                return parsed
        except (SyntaxError, ValueError):
            pass
    # If it's not a string or couldn't parse, return an empty list or None
    return []

df_pd['production_countries'] = df_pd['production_countries'].apply(parse_as_list)
df_pd.head()
# mlb = MultiLabelBinarizer()
# countries_onehot = mlb.fit_transform(df_pd['production_countries'])


,id,title,original_title,release_date,popularity,vote_average,vote_count,budget,revenue,runtime,...,Actor_6_name,Actor_6_popularity,Actor_7_name,Actor_7_popularity,Actor_8_name,Actor_8_popularity,Actor_9_name,Actor_9_popularity,Actor_10_name,Actor_10_popularity
0,98,Gladiator,Gladiator,2000-05-04,5.174,8.218,19470,103000000,465361176,155,...,Djimon Hounsou,0.040,Spencer Treat Clark,0.037,Ralf Moeller,0.034,Omid Djalili,0.031,Sven-Ole Thorsen,0.029
1,77,Memento,Memento,2000-10-11,4.971,8.176,15144,9000000,40047236,113,...,Mark Boone Junior,0.037,Callum Keith Rennie,0.035,Jorja Fox,0.033,Marianne Muellerleile,0.029,Harriet Sansom Harris,0.025
2,9741,Unbreakable,Unbreakable,2000-11-22,4.907,7.100,9389,75000000,248118121,106,...,Eamonn Walker,0.029,Damian Young,0.026,Chance Kelly,0.025,James Handy,0.025,Leslie Stefanson,0.020
3,107,Snatch,Snatch,2000-09-01,4.870,7.814,9205,10000000,83600000,103,...,Rade Šerbedžija,0.035,Vinnie Jones,0.035,Alan Ford,0.028,Lennie James,0.028,Ewen Bremner,0.023
4,9532,Final Destination,Final Destination,2000-03-17,4.200,6.603,5757,23000000,112880294,98,...,Alessandro Juliani,0.026,Kerr Smith,0.025,Brendan Fehr,0.022,Christine Chatelain,0.021,P. Lynn Johnson,0.021


In [23]:
print(type(df_pd['production_countries'].iloc[0]))
# Should show <class 'list'>
print(df_pd['production_countries'].iloc[0])
# Check the type of the first element
print(type(df_pd['production_countries'].iloc[0][0]))
# Should show <class 'str'> if they're strings

mlb = MultiLabelBinarizer()
countries_onehot = mlb.fit_transform(df_pd['production_countries'])

print(mlb.classes_)  # Should now show actual country names

<class 'list'>
['United Kingdom', 'United States of America']
<class 'str'>
['Afghanistan' 'Albania' 'Algeria' 'Andorra' 'Angola' 'Anguilla'
 'Antarctica' 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Aruba'
 'Australia' 'Austria' 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh'
 'Barbados' 'Belarus' 'Belgium' 'Belize' 'Benin' 'Bermuda' 'Bhutan'
 'Bolivia' 'Bosnia and Herzegovina' 'Botswana' 'Brazil'
 'British Indian Ocean Territory' 'Brunei Darussalam' 'Bulgaria'
 'Burkina Faso' 'Burundi' 'Cambodia' 'Cameroon' 'Canada' 'Cayman Islands'
 'Central African Republic' 'Chad' 'Chile' 'China' 'Christmas Island'
 'Cocos  Islands' 'Colombia' 'Congo' 'Cook Islands' 'Costa Rica'
 "Cote D'Ivoire" 'Croatia' 'Cuba' 'Cyprus' 'Czech Republic' 'Denmark'
 'Djibouti' 'Dominica' 'Dominican Republic' 'Ecuador' 'Egypt'
 'El Salvador' 'Equatorial Guinea' 'Eritrea' 'Estonia' 'Ethiopia'
 'Faeroe Islands' 'Fiji' 'Finland' 'France' 'French Polynesia' 'Gabon'
 'Gambia' 'Georgia' 'Germany' 'Ghana' 'Greece' 'Greenland

In [24]:
# Create a DataFrame of one-hot columns
new_columns = ["country_" + str(country) for country in mlb.classes_]
countries_df = pd.DataFrame(countries_onehot, columns=new_columns, index=df_pd.index)
countries_df.head()
# Merge back with the original DataFrame (and drop the old column if you want)
df_pd = pd.concat([df_pd, countries_df], axis=1).drop(columns=['production_countries'])


In [31]:
# Step 4: Identify numeric and categorical columns
target_col = 'vote_average'
X = df_pd.drop(columns=[target_col])
y = df_pd[target_col]

numeric_features = X.select_dtypes(include=[np.number, 'bool']).columns.tolist()

categorical_features = X.select_dtypes(include=['object']).columns.tolist()

exclude_cols = ['id', 'movie_id','vote_count','revenue','popularity']
numeric_features = [col for col in numeric_features if col not in exclude_cols]

include_cols = []
categorical_features = [col for col in categorical_features if col in include_cols]
categorical_features

columns_to_keep = numeric_features + categorical_features
X = X[columns_to_keep]
X.head()

,budget,runtime,adult,video,genre_Drama,genre_Documentary,genre_Comedy,genre_Horror,genre_Thriller,genre_Romance,...,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Vietnam,country_Western Sahara,country_Yemen,country_Yugoslavia,country_Zambia,country_Zimbabwe
0,103000000,155,False,False,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9000000,113,False,False,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,75000000,106,False,False,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,10000000,103,False,False,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,23000000,98,False,False,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
# 3. Preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', StandardScaler())
])
# No categorical features needed for this scenario
# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

# Removing the categorical transformer from the ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    # ('cat', categorical_transformer, categorical_features) # Removed this line
],
    remainder='passthrough' # Added this to handle remaining columns
)

In [34]:
# 5. Split data
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

In [35]:
X_train

,budget,runtime,adult,video,genre_Drama,genre_Documentary,genre_Comedy,genre_Horror,genre_Thriller,genre_Romance,...,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Vietnam,country_Western Sahara,country_Yemen,country_Yugoslavia,country_Zambia,country_Zimbabwe
81431,0,79,False,False,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2767,0,120,False,False,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
28384,0,0,False,False,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44795,0,71,False,False,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13643,0,85,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54886,0,5,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76820,0,101,False,False,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103694,0,110,False,False,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
860,0,86,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:

# 4. Full pipeline with LinearRegression
model = make_pipeline(
    preprocessor,
    LinearRegression()
)



# 6. Train
model.fit(X_train, y_train)




Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['budget', 'runtime', 'adult',
                                                   'video', 'genre_Drama',
                                                   'genre_Documentary',
                                                   'genre_Comedy',
                                                   'genre_Horror',
                                                   'genre_Thriller',
                                                   'genre_Romance',
                                                   'genre_Acti...
                                                   'genre_Mystery',
                                                   'genre_Adventure',
                                                   'company_BBC',
                                                   'company_Lifetime',
                                                   'company_Universal Pictures',
                                                   'company_Reel One '
                                                   'Entertainment',
                                                   'company_Warner Bros. '
                                                   'Pictures',
                                                   'company_HBO Documentary '
                                                   'Films',
                                                   'company_Lionsgate',
                                                   'company_MarVista '
                                                   'Entertainment',
                                                   'company_The Asylum',
                                                   'company_Paramount Pictures',
                                                   'language_English', ...])])),
                ('linearregression', LinearRegression())])

In [37]:
# 7. Evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Test MSE:", mse)
print("Test R^2:", r2)

y_pred_train = model.predict(X_train)
mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

print("Test MSE:", mse_train)
print("Test R^2:", r2_train)

Test MSE: 3.9919089548074775
Test R^2: 0.11281814340324436
Test MSE: 4.067109572749091
Test R^2: 0.11307254047823379


In [45]:
min(y_pred_train)

np.float64(-13.089706240852978)

In [38]:
feature_names = preprocessor.get_feature_names_out()

coefs = model.named_steps['linearregression'].coef_

feature_importance = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefs
})

# Optionally, sort by the absolute value of the coefficients for importance ranking
feature_importance['abs_coef'] = feature_importance['coefficient'].abs()
feature_importance = feature_importance.sort_values(by='abs_coef', ascending=False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
print(feature_importance[feature_importance['abs_coef']>0.01])

                                   feature  coefficient  abs_coef
5                   num__genre_Documentary      0.39484   0.39484
59                 num__movie_idcredits_df      0.35563   0.35563
11                        num__genre_Music      0.24495   0.24495
7                        num__genre_Horror     -0.21824   0.21824
13                    num__genre_Animation      0.15557   0.15557
0                              num__budget      0.11475   0.11475
1                             num__runtime     -0.10206   0.10206
29                   num__language_English     -0.07613   0.07613
17                      num__genre_Mystery      0.07492   0.07492
278            num__country_United Kingdom      0.07440   0.07440
4                         num__genre_Drama      0.07105   0.07105
69                num__Actor_10_popularity      0.06499   0.06499
12                     num__genre_TV Movie      0.05662   0.05662
8                      num__genre_Thriller     -0.05528   0.05528
60        

In [47]:
pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.6/169.6 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of category-encoders to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from pycaret.regression import *

In [12]:
import numpy as np
# Check if any columns are numpy array, if yes then I gonna drop
for col in df_pd.columns:
    if df_pd[col].apply(lambda x: isinstance(x, np.ndarray)).any():
        print(f"Column {col} contains numpy arrays!")

Column genres contains numpy arrays!
Column production_companies contains numpy arrays!
Column spoken_languages contains numpy arrays!


In [14]:
exclude_cols = ['id', 'movie_id','vote_count','revenue','popularity','genres','production_companies','spoken_languages']
df_pd_filtered = df_pd.drop(columns=exclude_cols)


In [ ]:
reg_setup = setup(df_pd_filtered, target="vote_average", session_id=42)  # ✅ For regression
rf_model = create_model("rf")
tuned_rf = tune_model(rf_model)



,Description,Value
0,Session id,42
1,Target,vote_average
2,Target type,Regression
3,Original data shape,"(108377, 113)"
4,Transformed data shape,"(108377, 113)"
5,Transformed train set shape,"(75863, 113)"
6,Transformed test set shape,"(32514, 113)"
7,Numeric features,68
8,Categorical features,42
9,Rows with missing values,100.0%


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.6499,4.7507,2.1796,-0.0097,0.3995,0.4755
1,1.6668,4.7935,2.1894,-0.0103,0.4031,0.4944
2,1.6594,4.7476,2.1789,-0.0040,0.3964,0.4836
3,1.6665,4.7710,2.1843,-0.0066,0.3997,0.4972
4,1.6663,4.7566,2.1810,-0.0003,0.3955,0.4833
5,1.6337,4.6601,2.1587,-0.0024,0.3971,0.4955
6,1.6444,4.6792,2.1631,-0.0036,0.3930,0.4800
7,1.6359,4.6498,2.1563,0.0036,0.3935,0.4813
8,1.6521,4.7672,2.1834,-0.0085,0.4017,0.4898


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,03:02:16
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [ ]:
plot_model(tuned_rf, plot="feature")  # Feature Importance

In [ ]:
evaluate_model(tuned_rf)  # Interactive GUI for evaluation


In [ ]:
predictions = predict_model(tuned_rf, data=df)
print(predictions.head())  # Includes predicted labels and probabilities
